# softmax回归
:label:`sec_softmax`

在 :numref:`sec_linear_regression`中我们介绍了线性回归。
随后，在 :numref:`sec_linear_scratch`中我们从头实现线性回归。
然后，在 :numref:`sec_linear_concise`中我们使用深度学习框架的高级API简洁实现线性回归。

回归可以用于预测*多少*的问题。
比如预测房屋被售出价格，或者棒球队可能获得的胜场数，又或者患者住院的天数。

事实上，我们也对*分类*问题感兴趣：不是问“多少”，而是问“哪一个”：

* 某个电子邮件是否属于垃圾邮件文件夹？
* 某个用户可能*注册*或*不注册*订阅服务？
* 某个图像描绘的是驴、狗、猫、还是鸡？
* 某人接下来最有可能看哪部电影？

通常，机器学习实践者用*分类*这个词来描述两个有微妙差别的问题：
1. 我们只对样本的“硬性”类别感兴趣，即属于哪个类别；
2. 我们希望得到“软性”类别，即得到属于每个类别的概率。
这两者的界限往往很模糊。其中的一个原因是：即使我们只关心硬类别，我们仍然使用软类别的模型。

## 分类问题
:label:`subsec_classification-problem`

我们从一个图像分类问题开始。
假设每次输入是一个$2\times2$的灰度图像。
我们可以用一个标量表示每个像素值，每个图像对应四个特征$x_1, x_2, x_3, x_4$。
此外，假设每个图像属于类别“猫”“鸡”和“狗”中的一个。

接下来，我们要选择如何表示标签。
我们有两个明显的选择：最直接的想法是选择$y \in \{1, 2, 3\}$，
其中整数分别代表$\{\text{狗}, \text{猫}, \text{鸡}\}$。
这是在计算机上存储此类信息的有效方法。
如果类别间有一些自然顺序，
比如说我们试图预测$\{\text{婴儿}, \text{儿童}, \text{青少年}, \text{青年人}, \text{中年人}, \text{老年人}\}$，
那么将这个问题转变为回归问题，并且保留这种格式是有意义的。

但是一般的分类问题并不与类别之间的自然顺序有关。
幸运的是，统计学家很早以前就发明了一种表示分类数据的简单方法：*独热编码*（one-hot encoding）。
独热编码是一个向量，它的分量和类别一样多。
类别对应的分量设置为1，其他所有分量设置为0。
在我们的例子中，标签$y$将是一个三维向量，
其中$(1, 0, 0)$对应于“猫”、$(0, 1, 0)$对应于“鸡”、$(0, 0, 1)$对应于“狗”：

$$y \in \{(1, 0, 0), (0, 1, 0), (0, 0, 1)\}.$$

## 网络架构

为了估计所有可能类别的条件概率，我们需要一个有多个输出的模型，每个类别对应一个输出。
为了解决线性模型的分类问题，我们需要和输出一样多的*仿射函数*（affine function）。
每个输出对应于它自己的仿射函数。
在我们的例子中，由于我们有4个特征和3个可能的输出类别，
我们将需要12个标量来表示权重（带下标的$w$），
3个标量来表示偏置（带下标的$b$）。
下面我们为每个输入计算三个*未规范化的预测*（logit）：$o_1$、$o_2$和$o_3$。

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
$$
or in matrix
$$
\begin{bmatrix}
o_1 \\ o_2 \\ o_3 
\end{bmatrix} = \begin{bmatrix}
w_{11} &  w_{12} & w_{13} \\ 
w_{21} &  w_{22} & w_{23} \\
w_{31} &  w_{32} & w_{33}
\end{bmatrix} \cdot
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 
\end{bmatrix}
+
\begin{bmatrix}
b_1 \\ b_2 \\ b_3 
\end{bmatrix}
$$

我们可以用神经网络图 :numref:`fig_softmaxreg`来描述这个计算过程。
与线性回归一样，softmax回归也是一个单层神经网络。
由于计算每个输出$o_1$、$o_2$和$o_3$取决于
所有输入$x_1$、$x_2$、$x_3$和$x_4$，
所以softmax回归的输出层也是全连接层。

![softmax回归是一种单层神经网络](../img/softmaxreg.svg)
:label:`fig_softmaxreg`

为了更简洁地表达模型，我们仍然使用线性代数符号。
通过向量形式表达为$\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$，
这是一种更适合数学和编写代码的形式。
由此，我们已经将所有权重放到一个$3 \times 4$矩阵中。
对于给定数据样本的特征$\mathbf{x}$，
我们的输出是由权重与输入特征进行矩阵-向量乘法再加上偏置$\mathbf{b}$得到的。

## 全连接层的参数开销
:label:`subsec_parameterization-cost-fc-layers`

正如我们将在后续章节中看到的，在深度学习中，全连接层无处不在。
然而，顾名思义，全连接层是“完全”连接的，可能有很多可学习的参数。
具体来说，对于任何具有$d$个输入和$q$个输出的全连接层，
参数开销为$\mathcal{O}(dq)$，这个数字在实践中可能高得令人望而却步。
幸运的是，将$d$个输入转换为$q$个输出的成本可以减少到$\mathcal{O}(\frac{dq}{n})$，
其中超参数$n$可以由我们灵活指定，以在实际应用中平衡参数节约和模型有效性
 :cite:`Zhang.Tay.Zhang.ea.2021`。

## softmax运算
:label:`subsec_softmax_operation`

现在我们将优化参数以最大化观测数据的概率。
为了得到预测结果，我们将设置一个**阈值**，如选择具有最大概率的标签。

我们希望模型的输出$\hat{y}_j$可以视为属于类$j$的概率，
然后选择具有最大输出值的类别$\operatorname*{argmax}_j y_j$作为我们的预测。
例如，如果$\hat{y}_1$、$\hat{y}_2$和$\hat{y}_3$分别为0.1、0.8和0.1，
那么我们预测的类别是2，在我们的例子中代表“鸡”。

然而我们能否将***未规范化的预测***$o$直接视作我们感兴趣的输出呢？
答案是否定的。
因为将线性层的输出直接视为概率时存在一些问题：
一方面，我们没有限制这些输出数字的总和为1。
另一方面，根据输入的不同，它们可以为负值。
这些违反了 :numref:`sec_prob`中所说的概率基本公理。

要将输出视为概率，**我们必须保证在任何数据上的输出都是非负的且总和为1**。
此外，我们需要一个训练的目标函数，来激励模型精准地估计概率。
例如，
在分类器输出0.5的所有样本中，我们希望这些样本是刚好有一半实际上属于预测的类别。
这个属性叫做*校准*（calibration）。

社会科学家邓肯·卢斯于1959年在*选择模型*（choice model）的理论基础上
发明的*softmax函数*正是这样做的：
***softmax函数能够将未规范化的预测变换为非负数并且总和为1，同时让模型保持
可导的性质***。
为了完成这一目标，我们首先对每个未规范化的预测求幂，这样可以确保输出非负。
为了确保最终输出的概率值总和为1，我们再让每个求幂后的结果除以它们的总和。如下式：

$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$
:eqlabel:`eq_softmax_y_and_o`

即将神经层的输出$
\begin{bmatrix}
o_1 \\ o_2 \\ o_3 
\end{bmatrix}$作为输入，再次输入到激活函数$\mathrm{softmax}$中，得到激活后满足概率论公理的概率输出$
\begin{bmatrix}
\hat{y}_1 \\ \hat{y}_2 \\ \hat{y}_3
\end{bmatrix}$，再对此概率输出进行极大似然估计，其中的最大值作为最有可能输出的归类或者概率判断。

这里，对于所有的$j$总有$0 \leq \hat{y}_j \leq 1$。
因此，$\hat{\mathbf{y}}$可以视为一个正确的概率分布。
softmax运算不会改变未规范化的预测$\mathbf{o}$之间的大小次序，只会确定分配给每个类别的概率。
因此，在预测过程中，我们仍然可以用下式来选择最有可能的类别。

$$
\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.
$$

尽管softmax是一个非线性函数，但softmax回归的输出仍然由输入特征的仿射变换决定。
因此，softmax回归是一个*线性模型*（linear model）。

## 小批量样本的矢量化
:label:`subsec_softmax_vectorization`

为了提高计算效率并且充分利用GPU，我们通常会对小批量样本的数据执行矢量计算。
假设我们读取了一个批量的样本$\mathbf{X}$，
其中特征维度（输入数量）为$d$，批量大小为$n$。
此外，假设我们在输出中有$q$个类别。
那么小批量样本的特征为$\mathbf{X} \in \mathbb{R}^{n \times d}$，
权重为$\mathbf{W} \in \mathbb{R}^{d \times q}$，
偏置为$\mathbf{b} \in \mathbb{R}^{1\times q}$。
softmax回归的矢量计算表达式为：

$$ \begin{aligned} \mathbf{O}_{1\times q} &= \mathbf{X}_{n \times d} \mathbf{W}_{d \times q} + \mathbf{b}_{1\times q}, \\ \hat{\mathbf{Y}}_{1\times q} & = \mathrm{softmax}(\mathbf{O}_{1\times q}). \end{aligned} $$
:eqlabel:`eq_minibatch_softmax_reg`

相对于一次处理一个样本，
小批量样本的矢量化加快了$\mathbf{X}和\mathbf{W}$的矩阵-向量乘法。
由于$\mathbf{X}$中的每一行代表一个数据样本，
那么softmax运算可以*按行*（rowwise）执行：
对于$\mathbf{O}$的每一行，我们先对所有项进行幂运算，然后通过求和对它们进行标准化。
在 :eqref:`eq_minibatch_softmax_reg`中，
$\mathbf{X} \mathbf{W} + \mathbf{b}$的求和会使用广播机制，
小批量的未规范化预测$\mathbf{O}$和输出概率$\hat{\mathbf{Y}}$
都是形状为$n \times q$的矩阵。

## 损失函数

接下来，我们需要一个损失函数来度量预测的效果。
我们将使用最大似然估计，这与在线性回归
（ :numref:`subsec_normal_distribution_and_squared_loss`）
中的方法相同。

### 对数似然

softmax函数给出了一个向量$\hat{\mathbf{y}}$，
我们可以将其视为“对给定任意输入$\mathbf{x}$的每个类的条件概率”。
例如，$\hat{y}_1$=$P(y=\text{猫} \mid \mathbf{x})$，即在给定输入 $\mathbf{x}$ 的前提下，类别是“猫”的可能性。


假设整个数据集$\{\mathbf{X}, \mathbf{Y}\}$具有$n$个样本，
其中索引$i$的样本由特征向量$\mathbf{x}^{(i)}$和独热标签向量$\mathbf{y}^{(i)}$组成。
我们可以将估计值与实际值进行比较：

$$
P(\mathbf{Y} \mid \mathbf{X}) = \prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}).
$$

- $P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})$ 表示：
  > “在给定输入 $\mathbf{x}^{(i)}$ 的条件下，真实标签恰好是 $\mathbf{y}^{(i)}$ 的概率”

由于 $\mathbf{y}^{(i)}$ 是 one-hot 向量（只有第 $k$ 位为 1），所以：
$$
P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}) = \hat{y}^{(i)}_k = P(y = k \mid \mathbf{x}^{(i)})
$$

> 💡 举例：  
> 若 $\mathbf{y}^{(i)} = [0, 1, 0]$（真实类别是“狗”），  
> 则 $P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}) = \hat{y}^{(i)}_2$（模型预测“狗”的概率）  
> $P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}) = \hat{y}^{(i)}_k$ 仅在$\mathbf{y}^{(i)}$ 是 one-hot 向量时成立。因为$\mathbf{y}^{(i)}$中只有$\mathbf{y}^{(k)} =1$，其余 $\mathbf{y}^{(j)} =0$，所以结果就是 $\hat{y}^{(i)}_k$

根据定义
- $\mathbf{X} = [\mathbf{x}^{(1)}, \dots, \mathbf{x}^{(n)}]$：所有输入
- $\mathbf{Y} = [\mathbf{y}^{(1)}, \dots, \mathbf{y}^{(n)}]$：所有标签

- **假设样本独立同分布（i.i.d.）** → 联合概率 = 各样本概率的乘积

所以：
$$
P(\mathbf{Y} \mid \mathbf{X}) = \text{“整个数据集被正确预测的概率”}
$$

> 🎯 **目标**：找到模型参数，使得这个联合概率**最大** → **最大似然估计（MLE）**

根据最大似然估计，我们最大化$P(\mathbf{Y} \mid \mathbf{X})$，相当于最小化负对数似然：

$$
-\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})
= \sum_{i=1}^n l(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}),
$$
---
为什么最大化 $P(\mathbf{Y} \mid \mathbf{X})$ 等价于最小化负对数似然？
原因有三：
1. **乘积难优化，求和易优化**
- 直接最大化 $\prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})$ 涉及大量小数相乘 → 数值下溢（underflow）
- 取对数将乘积变求和：
  $$
  \log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n \log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})
  $$

2. **优化方向转换**
- 最大化 $\log P(\mathbf{Y} \mid \mathbf{X})$  
  ⇨ 等价于  
  **最小化 $-\log P(\mathbf{Y} \mid \mathbf{X})$**

3. **得到标准损失函数形式**
代入 $P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}) = \hat{y}^{(i)}_{k_i}$（$k_i$ 是真实类别）：
$$
-\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log \hat{y}^{(i)}_{k_i}
$$

这正是**交叉熵损失**！

---

其中，对于任何标签$\mathbf{y}$和模型预测$\hat{\mathbf{y}}$，损失函数为：

$$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$
:eqlabel:`eq_l_cross_entropy`

在本节稍后的内容会讲到， :eqref:`eq_l_cross_entropy`中的损失函数
通常被称为*交叉熵损失*（cross-entropy loss）。
由于$\mathbf{y}$是一个长度为$q$的独热编码向量，
所以除了一个项以外的所有项$j$都消失了。
由于所有$\hat{y}_j$都是预测的概率，所以它们的对数永远不会大于$0$。
因此，如果正确地预测实际标签，即如果实际标签$P(\mathbf{y} \mid \mathbf{x})=1$，
则损失函数不能进一步最小化。
注意，这往往是不可能的。
例如，数据集中可能存在标签噪声（比如某些样本可能被误标），
或输入特征没有足够的信息来完美地对每一个样本分类。

### softmax及其导数
:label:`subsec_softmax_and_derivatives`

由于softmax和相关的损失函数很常见，
因此我们需要更好地理解它的计算方式。
将 :eqref:`eq_softmax_y_and_o`代入损失 :eqref:`eq_l_cross_entropy`中。
利用softmax的定义，我们得到：

$$
\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j\\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}
$$
最后一行等号依然是因为$y_j$ 是 one-hot 向量时成立，此时指标缩并，得到$\log \sum_{k=1}^q \exp(o_k)$

考虑相对于任何未规范化的预测$o_j$的导数，我们得到：

$$
\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j.
$$

换句话说，导数是我们softmax模型分配的概率与实际发生的情况（由独热标签向量表示）之间的差异。
从这个意义上讲，这与我们在回归中看到的非常相似，
其中梯度是观测值$y$和估计值$\hat{y}$之间的差异。
这不是巧合，在任何指数族分布模型中
（参见[本书附录中关于数学分布的一节](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/distributions.html)），
对数似然的梯度正是由此得出的。
这使梯度计算在实践中变得容易很多。

### 交叉熵损失

现在让我们考虑整个结果分布的情况，即观察到的不仅仅是一个结果。
对于标签$\mathbf{y}$，我们可以使用与以前相同的表示形式。
唯一的区别是，我们现在用一个概率向量表示，如$(0.1, 0.2, 0.7)$，
而不是仅包含二元项的向量$(0, 0, 1)$。
我们使用 :eqref:`eq_l_cross_entropy`来定义损失$l$，
它是所有标签分布的预期损失值。
此损失称为*交叉熵损失*（cross-entropy loss），它是分类问题最常用的损失之一。
本节我们将通过介绍信息论基础来理解交叉熵损失。
如果想了解更多信息论的细节，请进一步参考
[本书附录中关于信息论的一节](https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html)。

## 信息论基础
:label:`subsec_info_theory_basics`

*信息论*（information theory）涉及编码、解码、发送以及尽可能简洁地处理信息或数据。

### 熵

信息论的核心思想是量化数据中的信息内容。
在信息论中，该数值被称为分布$P$的*熵*（entropy）。可以通过以下方程得到：

$$H[P] = \sum_j - P(j) \log P(j).$$
:eqlabel:`eq_softmax_reg_entropy`

信息论的基本定理之一指出，为了对从分布$p$中随机抽取的数据进行编码，
我们至少需要$H[P]$“纳特（nat）”对其进行编码。
“纳特”相当于*比特*（bit），但是对数底为$e$而不是2。因此，一个纳特是$\frac{1}{\log(2)} \approx 1.44$比特。

---
**“对数据进行编码”是什么意思？**
- 指用**二进制串（或符号串）表示数据**，目标是**尽可能短**。
- 例如：
  - 天气有 4 种：晴、雨、阴、雪
  - 若等概率出现，可用 2 位编码：`00, 01, 10, 11`
  - 但如果“晴”占 90%，我们可以用更短码表示“晴”（如 `0`），长码表示其他（如 `10`, `110`, `111`）→ **变长编码**，此时由于“晴”最多，因此平均编码长度反而变短。

> 🎯 编码目标：**平均码长最短**

---


### 信息量

压缩与预测有什么关系呢？
想象一下，我们有一个要压缩的数据流。
如果我们很容易预测下一个数据，那么这个数据就很容易压缩。
为什么呢？
举一个极端的例子，假如数据流中的每个数据完全相同，这会是一个非常无聊的数据流。
由于它们总是相同的，我们总是知道下一个数据是什么。
所以，为了传递数据流的内容，我们不必传输任何信息。也就是说，“下一个数据是xx”这个事件毫无信息量。

但是，如果我们不能完全预测每一个事件，那么我们有时可能会感到"惊异"。
克劳德·香农决定用信息量$\log \frac{1}{P(j)} = -\log P(j)$来量化这种惊异程度。
在观察一个事件$j$时，并赋予它（主观）概率$P(j)$。
当我们赋予一个事件较低的概率时，我们的惊异会更大，该事件的信息量也就更大。
在 :eqref:`eq_softmax_reg_entropy`中定义的熵，
是当分配的概率真正匹配数据生成过程时的*信息量的期望*。

**香农信源编码定理（Shannon's Source Coding Theorem）** 指出：
> **对于独立同分布（i.i.d.）的随机变量序列，任何无损编码的平均码长 $L$ 满足：**
$L \geq H[p]$
> **且存在编码方案使 $L$ 任意接近 $H[p]$。**

即：
- **$H[p]$ 是无损压缩的“理论下限”**
- 你**不可能**设计出平均码长小于 $H[p]$ 的无损编码
- 但你可以**无限逼近**这个极限（如霍夫曼编码、算术编码）

---
场景：编码天气预报
- 分布：$p(\text{晴}) = 0.8,\ p(\text{雨}) = 0.2$
- 熵（nat）：
  $$
  H = -[0.8 \ln 0.8 + 0.2 \ln 0.2] \approx 0.5\ \text{nat}
  $$

### 编码方案对比：
| 方案 | 编码 | 平均码长（nat） | 是否可行 |
|------|------|------------------|--------|
| 固定长度 | 晴=`0`, 雨=`1` | $1\ \text{nat}$（若用 $\ln 2$ 作单位） | 可行，但非最优 |
| 最优编码 | 晴=`0`, 雨=`10` | 接近 $0.5\ \text{nat}$ | ✅ 可逼近熵 |
| “超压缩” | 平均码长 $0.4\ \text{nat}$ | ❌ 不可能！违反信息论 |

---

为什么是“至少需要 $H[p]$ 纳特”？

##### 从“信息量”角度理解：
- **罕见事件携带更多信息**（如“火山爆发”比“今天晴”信息量大）
- 熵 $H[p]$ 是**每个符号的平均信息量**
- 要无损表示这些信息，**码长不能少于信息量本身**

---
与机器学习的联系

在深度学习中，**交叉熵损失**正是基于这一原理：
- 模型输出分布 $\hat{y}$ 应逼近真实分布 $y$
- 最小化交叉熵 $H(y, \hat{y})$ ⇨ 让模型用最少的“信息量”描述数据
- 当 $\hat{y} = y$ 时，交叉熵 = 熵 $H[y]$ → 达到理论最优

---
**“为了无损地表示从分布 $p$ 中产生的数据，每个样本平均所需的最小编码长度（以 nat 为单位）就是该分布的熵 $H[p]$。”**



### 重新审视交叉熵

如果把熵$H(P)$想象为“知道真实概率的人所经历的惊异程度”，那么什么是交叉熵？
交叉熵*从*$P$*到*$Q$，记为$H(P, Q)$。
我们可以把交叉熵想象为“主观概率为$Q$的观察者在看到根据概率$P$生成的数据时的预期惊异”。
当$P=Q$时，交叉熵达到最低。
在这种情况下，从$P$到$Q$的交叉熵是$H(P, P)= H(P)$。

简而言之，我们可以从两方面来考虑交叉熵分类目标：
（i）最大化观测数据的似然；（ii）最小化传达标签所需的惊异。

## 模型预测和评估

在训练softmax回归模型后，给出任何样本特征，我们可以预测每个输出类别的概率。
通常我们使用预测概率最高的类别作为输出类别。
如果预测与实际类别（标签）一致，则预测是正确的。
在接下来的实验中，我们将使用*精度*（accuracy）来评估模型的性能。
精度等于正确预测数与预测总数之间的比率。

## 小结

* softmax运算获取一个向量并将其映射为概率。
* softmax回归适用于分类问题，它使用了softmax运算中输出类别的概率分布。
* 交叉熵是一个衡量两个概率分布之间差异的很好的度量，它测量给定模型编码数据所需的比特数。

## 练习

1. 我们可以更深入地探讨指数族与softmax之间的联系。
    1. 计算softmax交叉熵损失$l(\mathbf{y},\hat{\mathbf{y}})$的二阶导数。
    1. 计算$\mathrm{softmax}(\mathbf{o})$给出的分布方差，并与上面计算的二阶导数匹配。
1. 假设我们有三个类发生的概率相等，即概率向量是$(\frac{1}{3}, \frac{1}{3}, \frac{1}{3})$。
    1. 如果我们尝试为它设计二进制代码，有什么问题？
    1. 请设计一个更好的代码。提示：如果我们尝试编码两个独立的观察结果会发生什么？如果我们联合编码$n$个观测值怎么办？
1. softmax是对上面介绍的映射的误称（虽然深度学习领域中很多人都使用这个名字）。真正的softmax被定义为$\mathrm{RealSoftMax}(a, b) = \log (\exp(a) + \exp(b))$。
    1. 证明$\mathrm{RealSoftMax}(a, b) > \mathrm{max}(a, b)$。
    1. 证明$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \mathrm{max}(a, b)$成立，前提是$\lambda > 0$。
    1. 证明对于$\lambda \to \infty$，有$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$。
    1. soft-min会是什么样子？
    1. 将其扩展到两个以上的数字。

[Discussions](https://discuss.d2l.ai/t/1785)


1. 我们可以更深入地探讨指数族与softmax之间的联系。
    1. 计算softmax交叉熵损失$l(\mathbf{y},\hat{\mathbf{y}})$的二阶导数。
    1. 计算$\mathrm{softmax}(\mathbf{o})$给出的分布方差，并与上面计算的二阶导数匹配。

---
Softmax 输出的概率分布属于 **指数族分布（Exponential Family）**。对于分类问题，其自然参数是 logits $\mathbf{o}$，充分统计量是 one-hot 向量。

我们考虑一般标签分布 $\mathbf{y} \in \Delta^{q-1}$（概率 simplex，不要one-hot），损失为：
$$
l(\mathbf{y}, \hat{\mathbf{y}}) = -\sum_{j=1}^q y_j \log \hat{y}_j, \quad \text{其中 } \hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})
$$

---
(a) 计算 softmax 交叉熵损失的二阶导数（Hessian）

已知：
$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o})\quad \text{其中}\quad \hat{y}_j = \frac{\exp(o_j)}{\sum_k \exp(o_k)}$$
$$l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j$$



一阶导数（梯度）：
$$
\frac{\partial}{\partial o_i} l(\mathbf{y}, \hat{\mathbf{y}}) = \hat{y}_i - y_i
$$
（即使 $\mathbf{y}$ 不是 one-hot，此式仍成立！）

二阶导数（Hessian 矩阵 $H$）：
$$
H_{ij} = \frac{\partial^2 }{\partial o_i \partial o_j} l(\mathbf{y}, \hat{\mathbf{y}})
= \frac{\partial}{\partial o_j} (\hat{y}_i - y_i)
= \frac{\partial \hat{y}_i}{\partial o_j}
$$

而 softmax 的 Jacobian 为：
$$
\frac{\partial \hat{y}_i}{\partial o_j} =
\begin{cases}
\hat{y}_i (1 - \hat{y}_i), & i = j \\
- \hat{y}_i \hat{y}_j, & i \ne j
\end{cases}
$$

if $i \neq j$
$$
\begin{aligned}
\frac{\partial \hat{y}_i}{\partial o_j} 
&=\frac{\partial }{\partial o_j}\frac{\exp(o_j)}{\sum_k \exp(o_k)} \\
&= \frac{\frac{\partial }{\partial o_j}\exp(o_j)}{\sum_k \exp(o_k)} + \frac{\exp(o_j)}{\frac{\partial }{\partial o_j}\sum_k \exp(o_k)} \\
& = \frac{\exp(o_j)}{\sum_k \exp(o_k)}  +\frac{\exp(o_j) \cdot (-\exp(o_j) )}{(\sum_k \exp(o_k))^2} \\
& = \frac{\exp(o_j)}{\sum_k \exp(o_k)} \left(1- \frac{\exp(o_j) }{\sum_k \exp(o_k)} \right) \\
& = \hat{y}_i (1 - \hat{y}_i)
\end{aligned}
$$

if $i = j$
$$
\begin{aligned}
\frac{\partial \hat{y}_i}{\partial o_j} 
&=\frac{\partial }{\partial o_j}\frac{\exp(o_i)}{\sum_k \exp(o_k)} \\
&=\frac{\exp(o_j)}{\frac{\partial }{\partial o_i}\sum_k \exp(o_k)} \\
&=\frac{\exp(o_j) \cdot (-\exp(o_i) )}{(\sum_k \exp(o_k))^2} \\
&= - \hat{y}_i \hat{y}_j
\end{aligned}
$$

所以 Hessian 为：
$$
H = \mathrm{diag}(\hat{\mathbf{y}}) - \hat{\mathbf{y}} \hat{\mathbf{y}}^\top
$$

> ✅ 注意：**这个 Hessian 与 $\mathbf{y}$ 无关！**  
> 因为二阶导只依赖于模型输出 $\hat{\mathbf{y}}$，不依赖真实标签 $\mathbf{y}$。

---
(b) 计算 softmax 分布的协方差，并与 Hessian 匹配

设随机变量 $Z$ 表示类别（取值 $1,\dots,q$），其分布为 $P(Z = j) = \hat{y}_j$。

定义随机向量 $\mathbf{e}_Z \in \mathbb{R}^q$，则：
- $\mathbb{E}[\mathbf{e}_Z] = \hat{\mathbf{y}}$
- 协方差矩阵：
  $$
  \mathrm{Cov}(\mathbf{e}_Z) = \mathbb{E}[\mathbf{e}_Z \mathbf{e}_Z^\top] - \mathbb{E}[\mathbf{e}_Z]\mathbb{E}[\mathbf{e}_Z]^\top
  = \mathrm{diag}(\hat{\mathbf{y}}) - \hat{\mathbf{y}} \hat{\mathbf{y}}^\top
  $$

✅ **完全等于 Hessian！**

> 🎯 结论：  
> **交叉熵损失关于 logits 的 Hessian = softmax 输出分布的协方差矩阵**  
> 这是指数族分布的通用性质：**负对数似然的 Hessian = 充分统计量的协方差**



1. 假设我们有三个类发生的概率相等，即概率向量是$(\frac{1}{3}, \frac{1}{3}, \frac{1}{3})$。
    1. 如果我们尝试为它设计二进制代码，有什么问题？
    1. 请设计一个更好的代码。提示：如果我们尝试编码两个独立的观察结果会发生什么？如果我们联合编码$n$个观测值怎么办？

---
(a) 用二进制代码直接编码单个符号有什么问题？
- 每个符号需至少 $\log_2 3 \approx 1.585$ 比特
- 但**二进制码长必须是整数**（如 1 或 2 位）
  - 若用 1 位：只能表示 2 个符号 → ❌ 不够
  - 若用 2 位：平均码长 = 2 比特 > 1.585 → **效率低（冗余）**

> 💡 问题：**无法达到熵极限 $H = \log_2 3$**，因为码长离散。

(b) 更好的编码：联合编码多个观测值

思路：**利用大数定律，对 $n$ 个独立样本联合编码**

- $n$ 个样本的联合分布：均匀分布在 $3^n$ 个序列上
- 熵：$n \log_2 3$ 比特
- 可用 $\lceil n \log_2 3 \rceil$ 比特编码所有可能序列

#### 平均码长 per symbol：
$$
\frac{\lceil n \log_2 3 \rceil}{n} \to \log_2 3 \quad \text{as } n \to \infty
$$

✅ **渐近达到熵极限！**

> 📌 实际方法：**算术编码（Arithmetic Coding）** 或 **Tunstall 编码**  
> 它们能实现非整数码长的“效果”，逼近理论极限。

1. softmax是对上面介绍的映射的误称（虽然深度学习领域中很多人都使用这个名字）。真正的softmax被定义为$\mathrm{RealSoftMax}(a, b) = \log (\exp(a) + \exp(b))$。
    1. 证明$\mathrm{RealSoftMax}(a, b) > \mathrm{max}(a, b)$。
    1. 证明$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \mathrm{max}(a, b)$成立，前提是$\lambda > 0$。
    1. 证明对于$\lambda \to \infty$，有$\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$。
    1. soft-min会是什么样子？
    1. 将其扩展到两个以上的数字。

---

定义：
$$
\mathrm{RealSoftMax}(a, b) = \log(\exp(a) + \exp(b))
$$

这是 **log-sum-exp (LSE)** 函数，常用于数值稳定计算。

(a) 证明 $\mathrm{RealSoftMax}(a, b) > \max(a, b)$

设 $m = \max(a, b)$，则：
$$
\exp(a) + \exp(b) > \exp(m) \quad \text{（因为另一项 > 0）}
$$
取对数：
$$
\log(\exp(a) + \exp(b)) > \log(\exp(m)) = m = \max(a, b)
$$
✅ 得证。


(b) 证明 $\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \max(a, b)$ for $\lambda > 0$

由 (a)：
$$
\mathrm{RealSoftMax}(\lambda a, \lambda b) > \max(\lambda a, \lambda b) = \lambda \max(a, b)
$$
两边除以 $\lambda > 0$：
$$
\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) > \max(a, b)
$$
✅ 得证。


(c) 证明 $\lim_{\lambda \to \infty} \lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b) = \max(a, b)$

不失一般性，设 $a \geq b$，则：
$$
\mathrm{RealSoftMax}(\lambda a, \lambda b) = \log(\exp(\lambda a) + \exp(\lambda b)) \\
= \lambda a + \log(1 + \exp(\lambda (b - a)))
$$
因 $b - a \leq 0$，当 $\lambda \to \infty$，$\exp(\lambda (b - a)) \to 0$，所以：
$$
\lambda^{-1} \mathrm{RealSoftMax}(\lambda a, \lambda b)\\
= a + \lambda^{-1} \log(1 + \exp(\lambda (b - a))) \to a  \\
= \max(a, b)
$$
✅ 得证。

> 🎯 这说明：**log-sum-exp 是 max 函数的光滑近似**，$\lambda$ 控制“软度”。



(d) Soft-min 是什么？

类似地，定义：
$$
\mathrm{SoftMin}(a, b) = -\mathrm{RealSoftMax}(-a, -b) = -\log(\exp(-a) + \exp(-b))
$$

验证：
$$
\lim_{\lambda \to \infty} \lambda^{-1} \mathrm{SoftMin}(\lambda a, \lambda b) = \min(a, b)
$$



(e) 扩展到多个数

对向量 $\mathbf{x} = (x_1, \dots, x_n)$：

- **RealSoftMax（Log-Sum-Exp）**：
  $$
  \mathrm{LSE}(\mathbf{x}) = \log\left( \sum_{i=1}^n \exp(x_i) \right)
  $$
- **Softmax（概率归一化）**：
  $$
  \mathrm{softmax}_i(\mathbf{x}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)} = \nabla_{x_i} \mathrm{LSE}(\mathbf{x})
  $$
- **Soft-min**：
  $$
  \mathrm{SoftMin}(\mathbf{x}) = -\mathrm{LSE}(-\mathbf{x})
  $$

> 💡 在深度学习中，“softmax” 实际指 **归一化后的概率**，而 “RealSoftMax” 是其对数归一化因子（log-partition function）。




## **softmax 与 sigmoid 的关系**
我们可以将 **softmax 与 sigmoid 的关系** 理解为：

> **Softmax 是多分类的通用形式，而 Sigmoid 是 softmax 在二分类（两类）情况下的等价特例**。

但要注意一个**关键细节**：这种等价性成立的前提是——**二分类使用“单输出 + sigmoid” vs “双输出 + softmax”** 的两种不同建模方式。下面我们详细展开。

---

## ✅ 一、标准设定对比

### 情况 1：**二分类用 Sigmoid（常见做法）**
- 模型输出：**1 个 logit** $o \in \mathbb{R}$
- 预测概率：
  $$
  P(y=1) = \sigma(o) = \frac{1}{1 + e^{-o}}, \quad P(y=0) = 1 - \sigma(o)
  $$
- 损失函数：**Binary Cross-Entropy (BCE)**

### 情况 2：**二分类用 Softmax（视为 2 类多分类）**
- 模型输出：**2 个 logits** $\mathbf{o} = [o_0, o_1]$
- 预测概率：
  $$
  P(y=0) = \frac{e^{o_0}}{e^{o_0} + e^{o_1}}, \quad
  P(y=1) = \frac{e^{o_1}}{e^{o_0} + e^{o_1}}
  $$
- 损失函数：**Categorical Cross-Entropy**

---

## ✅ 二、两者何时等价？

### 关键观察：
如果我们在 softmax 的二分类中**强制让其中一个 logit 为 0**（或固定偏移），那么它就退化为 sigmoid！

#### 数学推导：

设 softmax 的两个 logits 为 $[0, o]$（即 $o_0 = 0, o_1 = o$），则：

$$
P(y=1) = \frac{e^{o}}{e^{0} + e^{o}} = \frac{e^{o}}{1 + e^{o}} = \frac{1}{1 + e^{-o}} = \sigma(o)
$$

✅ 完全等于 sigmoid！

> 💡 这说明：**sigmoid 相当于隐式地将负类 logit 固定为 0，只学习正类的 logit**。

---

## ✅ 三、为什么实践中常用 sigmoid 而不是 2-output softmax？

| 原因 | 说明 |
|------|------|
| **参数更少** | sigmoid 只需 1 个输出神经元；softmax 需 2 个（冗余） |
| **计算更省** | 少一半计算量和内存 |
| **习惯与接口** | PyTorch 的 `BCEWithLogitsLoss` 直接支持单输出 + sigmoid |

> 📌 注意：如果你用 **2-output softmax 做二分类**，其实是在做**冗余建模**，因为：
> $$
> [o_0, o_1] \quad \text{和} \quad [o_0 + c, o_1 + c]
> $$
> 经过 softmax 后结果完全相同（平移不变性）。所以实际只有 **1 个自由度**，和 sigmoid 一致。

---

## ✅ 四、多标签 vs 多类别：另一个重要区别

| 场景 | 输出数 | 激活函数 | 概率约束 |
|------|-------|--------|--------|
| **多类别分类（Mutually Exclusive）** | $K$ 类 | **Softmax** | $\sum p_i = 1$ |
| **多标签分类（Multi-label）** | $K$ 个独立标签 | **Sigmoid（每个输出独立）** | 每个 $p_i \in [0,1]$，无总和约束 |

> 🔥 例如：一张图可以同时有“猫”和“狗” → 用 sigmoid  
> 一张图只能是“猫”、“狗”或“鸟” → 用 softmax

所以，**sigmoid 不只是“二分类的 softmax”，更是“独立二分类”的基础**。

---

## ✅ 五、PyTorch 中的体现

```python
# 二分类：sigmoid 方式（推荐）
logit = torch.randn(4, 1)          # shape (N, 1)
prob = torch.sigmoid(logit)        # P(y=1)
loss = F.binary_cross_entropy(prob, target)

# 二分类：softmax 方式（等价但冗余）
logits = torch.randn(4, 2)         # shape (N, 2)
probs = F.softmax(logits, dim=1)   # [P(y=0), P(y=1)]
loss = F.cross_entropy(logits, target.long())  # target: 0 or 1
```

> ⚠️ 注意：`cross_entropy` 内部已经包含 softmax，不要自己加！

---

## ✅ 总结：你的理解是否正确？

| 说法 | 正确性 | 修正/补充 |
|------|--------|----------|
| “softmax 退化到二分类就是 sigmoid” | ✅ **基本正确** | 更准确说：**“当 softmax 用于二分类且固定一个 logit 时，等价于 sigmoid”** |
| “sigmoid 是 softmax 的特例” | ✅ 是 | 但仅限于**互斥二分类**场景 |
| “所有二分类都可用 softmax 替代 sigmoid” | ⚠️ 技术上可行，但不推荐 | 参数冗余，无必要 |

> 🎯 **最终结论**：  
> **是的，你可以将 sigmoid 视为 softmax 在二分类问题中的简化、等效形式**。  
> 这体现了机器学习中“统一框架下特例”的优美结构。